In [1]:
import numpy as np
import scipy.linalg as spla
import matplotlib.pyplot as plt
import dask.array as da
import dask
from dask.distributed import Client, LocalCluster
import verde as vd

In [2]:
client = Client(
    LocalCluster(n_workers=1, threads_per_worker=6, memory_limit="120Gb")
)
client

Client Scheduler: tcp://127.0.0.1:35337 Dashboard: http://127.0.0.1:8787/status,Cluster Workers: 1 Cores: 6 Memory: 120.00 GB


In [15]:
data = vd.datasets.CheckerBoard().scatter(size=10752)
east, north = data.easting.values, data.northing.values
coordinates = (east, north)
d = data.scalars + np.random.normal(scale=1, size=data.shape[0])
data

,northing,easting,scalars
0,-2847.952432,2744.067520,369.215891
1,-3371.786911,3575.946832,-246.224800
2,-4925.285531,3013.816880,944.315102
3,-1302.862432,2724.415915,-529.875663
4,-2798.950919,2118.273997,-598.382342
...,...,...,...
10747,-50.684791,1354.922313,-258.541077
10748,-2555.119864,925.506532,721.121313
10749,-3970.175862,4404.530777,848.420161
10750,-690.430666,3449.865820,-112.104049


In [16]:
def scale(x):
    scale_ = np.nanstd(x, axis=0)
    return x/scale_

def square_chunk(size, chunk):
    c = 1
    for n in range(1, size):
        if size % n == 0:
            c = n
        if n > 1*chunk:
            print("too big")
            break
    return c

def skinny_chunk(size, chunk):
    c = size*chunk//chunk**2
    return (size, size//c)


In [17]:
mindist = 100e3
damp = 1e-3
size = d.size
chunk = 1500
new_chunk = square_chunk(size, chunk)
#new_chunk = skinny_chunk(size, chunk)
print(new_chunk)

too big
1344


In [21]:
%%time
A = scale(vd.Spline(mindist=mindist).jacobian(coordinates, coordinates))
p = spla.solve(A.T@A + damp*np.eye(size), A.T@d, sym_pos=True)

CPU times: user 40.7 s, sys: 1.16 s, total: 41.8 s
Wall time: 39.6 s


In [18]:
def dask_green(e, n, md):
    d = da.sqrt(e**2 + n**2) + md
    return d**2 * (da.log(d) - 1)

def jacobian_dask(e, n, pe, pn, md):
    A = dask_green(
        e[np.newaxis].T - pe,
        n[np.newaxis].T - pn,
        md
    )
    return A

def scale_dask(x):
    scale_ = da.nanstd(x, axis=0)
    return x/scale_, scale_

In [19]:
deast, dnorth, dd = [
    da.from_array(i, chunks=new_chunk) for i in (east, north, d)
]

In [20]:
%%time
B, scale_ = scale_dask(jacobian_dask(deast, dnorth, deast, dnorth, mindist))
hessian = B.T@B 
gradient = B.T@dd
gradient = gradient.rechunk(new_chunk)
hessian = hessian.rechunk(new_chunk)
hessian = hessian + damp*da.eye(size, chunks=new_chunk)
#client.persist((hessian, gradient, B))
pa = da.linalg.solve(hessian, gradient).compute()
del hessian, gradient, B

CPU times: user 7.09 s, sys: 411 ms, total: 7.5 s
Wall time: 18.3 s


In [26]:
client.shutdown()

distributed.client - ERROR - Failed to reconnect to scheduler after 10.00 seconds, closing client
distributed.utils - ERROR - 
Traceback (most recent call last):
  File "/home/leo/miniconda3/envs/verde/lib/python3.7/site-packages/distributed/utils.py", line 662, in log_errors
    yield
  File "/home/leo/miniconda3/envs/verde/lib/python3.7/site-packages/distributed/client.py", line 1290, in _close
    await gen.with_timeout(timedelta(seconds=2), list(coroutines))
concurrent.futures._base.CancelledError
distributed.utils - ERROR - 
Traceback (most recent call last):
  File "/home/leo/miniconda3/envs/verde/lib/python3.7/site-packages/distributed/utils.py", line 662, in log_errors
    yield
  File "/home/leo/miniconda3/envs/verde/lib/python3.7/site-packages/distributed/client.py", line 1019, in _reconnect
    await self._close()
  File "/home/leo/miniconda3/envs/verde/lib/python3.7/site-packages/distributed/client.py", line 1290, in _close
    await gen.with_timeout(timedelta(seconds=2), l